<a href="https://colab.research.google.com/github/SaraElwatany/Lung-TumorDetection-Segmentation/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
from PIL import Image
from google.colab import drive
import matplotlib.pyplot as plt
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
drive.mount('/content/drive')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## **Helper Functions**

In [ ]:
def segment_tumor(image_path, model, detection=False):

   """
    Perform tumor segmentation on an input image using a trained model.

    This function supports two modes:
    - Detection mode: expects a small cropped RGB tumor image and resizes it to 64x64.
    - Full image segmentation mode: expects a grayscale image and resizes it to 256x256.

    Args:
        image_path (str): Path to the input image file.
        model (torch.nn.Module): Trained PyTorch model for tumor segmentation.
        detection (bool, optional): If True, perform inference on a cropped tumor patch.
                                    If False, perform segmentation on the full grayscale image. Default is False.

    Returns:
        None: Displays the input image and predicted mask using matplotlib.
    """


  # Perform inference on the cropped tumor
  if detection:

    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert to PIL
    image_pil = Image.fromarray(image)

    # Resize
    image_pil = image_pil.resize(64)

    # Convert to normalized tensor
    image = np.array(image_pil).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # HWC to CHW
    tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add batch dimension



  # Perform inference on the whole input image
  else:

    # Load image
    image = Image.open(image_path).convert("L")              # Convert to grayscale


    # Define transform (same as training)
    transform = T.Compose([
                            T.Resize((256, 256)),  # adjust to match training input size
                            T.ToTensor()
                        ])


    # Transform and add batch dimension
    input_tensor = transform(image).unsqueeze(0).to(device)         # Shape: [1, 1, H, W]



  # Forward pass
  with torch.no_grad():
      output = model(input_tensor)
      output = torch.sigmoid(output)      # Convert logits to probabilities
      pred_mask = (output > 0.5).float()  # Threshold for binary mask


  # Visualize
  plt.subplot(1, 3, 1)
  plt.title("Input Image")
  plt.imshow(image, cmap='gray')


  plt.subplot(1, 3, 3)
  plt.title("Predicted Mask")
  plt.imshow(pred_mask.squeeze().cpu(), cmap='gray')

  plt.show()

## **Tumor Segmentation on Whole Image**

In [ ]:
def double_convolution(in_channels, out_channels):
    """
    In the original paper implementation, the convolution operations were
    not padded but we are padding them here. This is because, we need the
    output result size to be same as input size.
    """
    conv_op = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                            nn.ReLU(inplace=True),

                            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                            nn.ReLU(inplace=True)
                           )
    return conv_op

In [ ]:
class UNet(nn.Module):


    def __init__(self, num_classes):

        super(UNet, self).__init__()

        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)

        # Contracting path.

        # Each convolution is applied twice.
        self.down_convolution_1 = double_convolution(1, 64)
        self.down_convolution_2 = double_convolution(64, 128)
        self.down_convolution_3 = double_convolution(128, 256)
        self.down_convolution_4 = double_convolution(256, 512)
        self.down_convolution_5 = double_convolution(512, 1024)


        # Expanding path.
        self.up_transpose_1 = nn.ConvTranspose2d(
                                                in_channels=1024, out_channels=512,
                                                kernel_size=2,
                                                stride=2)

        # Below, `in_channels` again becomes 1024 as we are concatinating.
        self.up_convolution_1 = double_convolution(1024, 512)

        self.up_transpose_2 = nn.ConvTranspose2d(
                                                  in_channels=512, out_channels=256,
                                                  kernel_size=2,
                                                  stride=2)

        self.up_convolution_2 = double_convolution(512, 256)

        self.up_transpose_3 = nn.ConvTranspose2d(
                                                  in_channels=256, out_channels=128,
                                                  kernel_size=2,
                                                  stride=2)

        self.up_convolution_3 = double_convolution(256, 128)

        self.up_transpose_4 = nn.ConvTranspose2d(
                                                  in_channels=128, out_channels=64,
                                                  kernel_size=2,
                                                  stride=2)

        self.up_convolution_4 = double_convolution(128, 64)

        # output => `out_channels` as per the number of classes.
        self.out = nn.Conv2d(
                              in_channels=64, out_channels=num_classes,
                              kernel_size=1
                            )



    def forward(self, x):

        down_1 = self.down_convolution_1(x)
        down_2 = self.max_pool2d(down_1)
        down_3 = self.down_convolution_2(down_2)
        down_4 = self.max_pool2d(down_3)
        down_5 = self.down_convolution_3(down_4)
        down_6 = self.max_pool2d(down_5)
        down_7 = self.down_convolution_4(down_6)
        down_8 = self.max_pool2d(down_7)
        down_9 = self.down_convolution_5(down_8)

        # *** DO NOT APPLY MAX POOL TO down_9 ***

        up_1 = self.up_transpose_1(down_9)
        x = self.up_convolution_1(torch.cat([down_7, up_1], 1))

        up_2 = self.up_transpose_2(x)
        x = self.up_convolution_2(torch.cat([down_5, up_2], 1))

        up_3 = self.up_transpose_3(x)
        x = self.up_convolution_3(torch.cat([down_3, up_3], 1))

        up_4 = self.up_transpose_4(x)
        x = self.up_convolution_4(torch.cat([down_1, up_4], 1))

        out = self.out(x)

        return out

In [ ]:
whole_img_model = UNet(num_classes=1).to(device)
whole_img_model.load_state_dict(torch.load(f"/content/drive/MyDrive/unet_lung_segmentation_0.002502174032005397.pth", map_location=device))
whole_img_model.to(device)

In [ ]:
segment_tumor_whole_img(image_path, true_mask_path, whole_img_model)

## **Tumor Detection**

In [ ]:
def detect_tumor():

  # Save a PDF file that has information about the existence/nonexistence of tumor, its location, size and number of fragments.
  pass

## **Segmentation of detected tumors**

In [ ]:
class DoubleConv(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.15),  # Slightly increased dropout
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

In [ ]:
class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, features=[32, 64, 128, 256]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()

        # Downsampling path (encoder)
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bottleneck
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)

        # Upsampling path (decoder)
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature * 2, feature))

        # Final output layer
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)  # transpose conv
            skip_connection = skip_connections[idx // 2]

            if x.shape != skip_connection.shape:
                x = F.interpolate(x, size=skip_connection.shape[2:])

            x = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](x)

        return self.final_conv(x)

In [ ]:
model_detection = UNet().to(device)
model_detection.load_state_dict(torch.load(f"/content/drive/MyDrive/unet_lung_segmentation_0.002502174032005397.pth", map_location=device))
model_detection.to(device)

In [ ]:
segment_tumor(image_path, true_mask_path, model_detection, detection=True)